In [ ]:
import os
from dotenv import dotenv_values 

# Load .env configuration
dotenv_path = r"c:\Users\Jakaria.Ahmed\OneDrive - insidemedia.net\Documents\cloud_backup\2025-Dev-Projects\Explore\nvidia-nim\.env"
env_config = dotenv_values(dotenv_path)
# print(env_config, "--",env_config["OPENAI_KEY"])

os.environ["env_openai_keys"] = env_config["OPENAI_KEY"]

# print(os.environ["env_openai_keys"])

$env_openai_keys
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


In [ ]:
nutritionist_system_prompt = """You are Dr. NutriGPT, a board-certified nutritionist with 15 years of clinical experience. 
Combine evidence-based guidelines from:
1. WHO dietary recommendations (2025)
2. NIH nutrient databases
3. Personalized health data from user's medical history
4. Latest research from PubMed (cutoff: Dec 2024)

Always:
- Request necessary biometrics before recommendations
- Cross-validate with user's cultural preferences
- Cite sources using [1][2] notation
- Provide meal plans with glycemic index considerations"""

In [ ]:
from openai import OpenAI
 
client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = os.environ["env_openai_keys"]
)
 
completion = client.chat.completions.create(
  model="meta/llama-3.3-70b-instruct",
  messages=[{"role":"user","content":"im having 5000 credits in my account, how many calls i can make using current model that is llama-3.3-70b-instruct via nvidia-nim"}],
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
  stream=True
)
 
for chunk in completion:
  if chunk.choices[0].delta.content is not None:
    print(chunk.choices[0].delta.content, end="")

To estimate how many calls you can make with 5000 credits using the LLaMA-3.3-70B-Instruct model via NVIDIA-NIM, we need to consider a few factors:

1. **Model Size and Complexity**: The LLaMA-3.3-70B-Instruct model is a large language model, with 70 billion parameters. Larger models generally require more computational resources and, by extension, more credits per call due to the increased computational cost.

2. **NVIDIA-NIM Pricing**: The cost per call can vary depending on the specific pricing model of NVIDIA-NIM. Pricing can be based on the model size, the length of the input, the number of tokens generated, or a combination of these factors. Without the exact pricing details, it's challenging to provide a precise estimate.

3. **Input and Output Size**: The number of tokens in your input and the number of tokens you request the model to generate also impact the cost. More tokens mean more computation, which translates to more credits used per call.

Given these variables, let's m

In [ ]:
from flask import Flask, request, Response
import os
from openai import OpenAI
from dotenv import dotenv_values

# Load .env configuration (adjust path as needed)
dotenv_path = r"c:\Users\Jakaria.Ahmed\OneDrive - insidemedia.net\Documents\cloud_backup\2025-Dev-Projects\Explore\nvidia-nim\.env"
env_config = dotenv_values(dotenv_path)
os.environ["env_openai_keys"] = env_config["OPENAI_KEY"]
# os.environ["env_openai_keys"] = env_config["OPENAI_KEY"]

app = Flask(__name__)

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = os.environ["env_openai_keys"]
)

nutritionist_system_prompt = """
You are Dr. NutriGPT, a board-certified nutritionist with 15 years of clinical experience.
Combine evidence-based guidelines from:
1. WHO dietary recommendations (2025)
2. NIH nutrient databases
3. Personalized health data from user's medical history
4. Latest research from PubMed (cutoff: Dec 2024)

Always:
- Request necessary biometrics before recommendations
- Cross-validate with user's cultural preferences
- Cite sources using [1][2] notation
- Provide meal plans with glycemic index considerations
Note:
- Use <br> for line breaks
-If returning any list, format the list using proper HTML structure like 
<b>Please share the following biometrics:</b>
<ul>
    <li><b>Age</b></li>
    <li><b>Sex</b></li>
    ...
 </ul>   
"""

@app.route("/query", methods=["POST"])
def query_model():
    user_input = request.json.get("input", "")

    # Create a streaming completion request
    completion = client.chat.completions.create(
        model="meta/llama-3.3-70b-instruct",  # Adjust to your chosen model
        messages=[
            {"role": "system", "content": nutritionist_system_prompt},
            {"role": "user", "content": user_input}
        ],
        temperature=0.2,
        top_p=0.7,
        max_tokens=1024,
        stream=True  # Enable streaming
    )

    def generate():
        try:
            for chunk in completion:
                if chunk.choices[0].delta and chunk.choices[0].delta.content:
                    text_piece = chunk.choices[0].delta.content
                    # Append <br> after each chunk for spacing
                    yield f"data: {text_piece}\n\n"
        except Exception as e:
            yield f"data: [Error] {str(e)}<br>\n\n"

    return Response(generate(), mimetype="text/event-stream")

if __name__ == "__main__":
    # Run in debug mode for development
    app.run(debug=True)


In [ ]:
from flask import Flask, request, Response, session
import os
from openai import OpenAI
from dotenv import dotenv_values
from flask_session import Session
from redis import Redis

# Load .env configuration (adjust path as needed)
dotenv_path = r"c:\Users\Jakaria.Ahmed\OneDrive - insidemedia.net\Documents\cloud_backup\2025-Dev-Projects\Explore\nvidia-nim\.env"
env_config = dotenv_values(dotenv_path)
os.environ["env_openai_keys"] = env_config["OPENAI_KEY"]
# os.environ["env_openai_keys"] = env_config["OPENAI_KEY"]

app = Flask(__name__)
app.secret_key = "super_secret_key" 

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = os.environ["env_openai_keys"]
)

nutritionist_system_prompt = """
You are Dr. NutriGPT, a board-certified nutritionist with 15 years of clinical experience.
Combine evidence-based guidelines from:
1. WHO dietary recommendations (2025)

Always:
- Cite sources using [1][2] notation
- Provide meal plans with glycemic index considerations
Note:
- see prvious messages for context
- Use <br> for line breaks
-If returning any list, format the list using proper HTML structure like 
<b>Please share the following biometrics:</b>
<ul>
    <li><b>Age</b></li>
    <li><b>Sex</b></li>
    ...
 </ul>   
"""

# 🔥 FIX: Store sessions in the filesystem instead of cookies
app.config["SESSION_TYPE"] = "filesystem"  # Use local file storage
app.config["SESSION_PERMANENT"] = False
app.config["SESSION_USE_SIGNER"] = True
app.config["SESSION_FILE_DIR"] = "./flask_session"  # Local storage directory

Session(app)  

@app.route("/query", methods=["POST"])
def query_model():
    user_input = request.json.get("input", "")

    # Ensure session messages exist
    if "messages" not in session:
        session["messages"] = []

    # Append user input to session messages
    session["messages"].append({"role": "user", "content": user_input})

    # Limit history (keep only the last 10 messages to prevent overflow)
    session["messages"] = session["messages"][-20:]
    session.modified = True  

    # Construct full message history for context retention
    messages = [{"role": "system", "content": nutritionist_system_prompt}] + session["messages"]

    # Debugging: Print messages before sending to model
    print("\n📌 Full Context Sent to Model:\n", messages, "\n")

    # Call OpenAI model with full context
    completion = client.chat.completions.create(
        model="meta/llama-3.3-70b-instruct",
        messages=messages,
        temperature=0.2,
        top_p=0.7,
        max_tokens=2024,
        stream=True
    )

    def generate():
        full_response = ""
        try:
            for chunk in completion:
                if chunk.choices[0].delta and chunk.choices[0].delta.content:
                    text_piece = chunk.choices[0].delta.content
                    full_response += text_piece
                    yield f"data: {text_piece}\n\n"

            # Store assistant response in session
            session["messages"].append({"role": "assistant", "content": full_response})

            # Print updated session for debugging
            print("\n✅ Updated Session Messages:\n", session["messages"], "\n")

        except Exception as e:
            yield f"data: [Error] {str(e)}<br>\n\n"

    return Response(generate(), mimetype="text/event-stream")

# New route to check session data
@app.route("/session-data", methods=["GET"])
def get_session_data():
    return {"session": session.get("messages", [])}

# New route to reset session
@app.route("/reset-session", methods=["POST"])
def reset_session():
    session.pop("messages", None)
    return {"message": "Session reset."}

if __name__ == "__main__":
    app.run(debug=True)




In [ ]:
import streamlit as st
import requests
import time
import re

# -------------------------------
# Session State Initialization
# -------------------------------
if "messages" not in st.session_state:
    st.session_state.messages = []

# Track usage stats
if "num_calls" not in st.session_state:
    st.session_state.num_calls = 0
if "total_input_tokens" not in st.session_state:
    st.session_state.total_input_tokens = 0
if "total_output_tokens" not in st.session_state:
    st.session_state.total_output_tokens = 0

# Set page config
st.set_page_config(
    page_title="AI Nutritionist Chatbot",
    page_icon="🥗",
    layout="wide"
)

# -------------------------------
# Custom CSS for Chat Layout
# -------------------------------
st.markdown("""
<style>
/* Light green background to indicate health */
body {
    background-color: #f2fff5; /* a very light, minty green */
    color: #333;
    font-family: "Helvetica", "Arial", sans-serif;
}

/* Center the main container for a simple look */
main.block-container {
    max-width: 900px;
    margin: 0 auto;
    padding: 1rem 2rem;
}

/* Chat messages styling */
.stChatMessage {
    border-radius: 8px;
    margin: 0.5rem 0;
    padding: 1rem;
    background-color: #ffffff; /* white bubble background */
    border: 1px solid #d2f2d2; /* subtle green border */
}

/* Different backgrounds for user vs. assistant messages */
.stChatMessage.user {
    background-color: #eaffea; /* slightly tinted green for user */
}
.stChatMessage.assistant {
    background-color: #ffffff; /* pure white for assistant */
}

/* Title styling */
h1 {
    color: #2F855A; /* a calm greenish color */
    margin-bottom: 0.2rem;
    text-align: left;
}

/* Subtitle styling */
.description {
    color: #555;
    font-size: 1rem;
    margin-bottom: 2rem;
}

/* Chat input styling */
.streamlit-chat-input > label {
    font-weight: bold;
    color: #2F855A;
}

/* Sidebar styling */
[data-testid="stSidebar"] {
    background-color: #f8fff9; /* a lighter greenish tint */
    border-right: 1px solid #d2f2d2;
}
[data-testid="stSidebar"] h2, [data-testid="stSidebar"] h3 {
    color: #2F855A;
}
</style>
""", unsafe_allow_html=True)

# -------------------------------
# Main Layout
# -------------------------------
st.title("AI Nutritionist Chatbot")
st.markdown(
    "<p class='description'>Ask questions about nutrition, meal plans, or dietary guidelines, "
    "and get personalized, evidence-based responses in a calm, health-inspired setting.</p>",
    unsafe_allow_html=True
)

# Display existing chat messages
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"], unsafe_allow_html=True)

# Chat input
prompt = st.chat_input("Ask a question about nutrition...")

if prompt:
    # Update usage stats
    st.session_state.num_calls += 1
    input_token_count = len(prompt.split())
    st.session_state.total_input_tokens += input_token_count

    # Convert user single newlines into <br> for line breaks
    user_text = prompt.replace("\n", "<br>")

    # Show user message in the chat with Markdown
    with st.chat_message("user"):
        st.markdown(user_text, unsafe_allow_html=True)

    st.session_state.messages.append({"role": "user", "content": user_text})

    # Prepare assistant message container
    with st.chat_message("assistant"):
        response_placeholder = st.empty()
        full_response = ""

        try:
            # Make streaming request to your Flask endpoint
            with requests.post(
                "http://127.0.0.1:5000/query",
                json={"input": prompt},
                stream=True
            ) as r:
                for line in r.iter_lines():
                    if line:
                        # SSE line typically starts with "data: "
                        if line.startswith(b"data: "):
                            chunk = line[6:].decode("utf-8", errors="ignore")
                            # Accumulate chunk
                            full_response += chunk
                            # Live "typing" effect
                            partial_display = full_response + "▌"
                            response_placeholder.markdown(partial_display, unsafe_allow_html=True)
                            time.sleep(0.01)

            # ---- Final re-render with spacing fixes ----
            # 1) Insert newlines before headings if missing
            final_text = re.sub(r'(?<!\n)(### )', r'\n\n\1', full_response)
            # 2) Convert single newlines to double newlines for paragraph breaks
            final_text = final_text.replace("\n", "\n\n")

            # Re-render the entire text without the blinking cursor
            response_placeholder.markdown(final_text, unsafe_allow_html=True)

            # (Optional) Count approximate output tokens
            output_token_count = len(full_response.split())
            st.session_state.total_output_tokens += output_token_count

            # Save final message
            st.session_state.messages.append({"role": "assistant", "content": final_text})

        except Exception as e:
            st.error(f"Error: {str(e)}")

# -------------------------------
# Sidebar / Right Panel
# -------------------------------
st.sidebar.header("Session Info")
st.sidebar.markdown(f"**Number of calls:** {st.session_state.num_calls}")
st.sidebar.markdown(f"**Total input tokens:** {st.session_state.total_input_tokens}")
st.sidebar.markdown(f"**Total output tokens:** {st.session_state.total_output_tokens}")

st.sidebar.markdown("---")
st.sidebar.markdown("**Model Used:** `meta/llama-3.3-70b-instruct`")


In [ ]:
# !pip install flask_session redis


Defaulting to user installation because normal site-packages is not writeable
